# Sentence Retrieval

In [ ]:
import json
import math
import nltk
from time import sleep
# from tqdm import tqdm

train_file = open("train.json",'r')
dev_file = open("dev.json",'r')
test_file=open("test.json",'r')
train = json.loads(train_file.read())
dev = json.loads(dev_file.read())
test = json.loads(test_file.read())

In [ ]:
import math
from six import iteritems
from six.moves import xrange

# BM25 parameters.
PARAM_K1 = 0.0
PARAM_B = 1.0
EPSILON = 0.0

class BM25(object):

    def __init__(self, corpus):
        self.corpus_size = len(corpus)
        self.avgdl = sum(map(lambda x: float(len(x)), corpus)) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.initialize()

    def initialize(self):
        for document in self.corpus:
            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in iteritems(frequencies):
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, document, index, average_idf):
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += (idf * self.f[index][word] * (PARAM_K1 + 1)
                      / (self.f[index][word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * self.corpus_size / self.avgdl)))
        return score

    def get_scores(self, document, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(document, index, average_idf)
            scores.append(score)
        return scores


def get_bm25_weights(corpus):
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

    weights = []
    for doc in corpus:
        scores = bm25.get_scores(doc, average_idf)
        weights.append(scores)
    return weights

In [ ]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
stemmer = nltk.stem.porter.PorterStemmer()

from nltk import StanfordPOSTagger

stanford_tagger = StanfordPOSTagger('english-bidirectional-distsim.tagger')

from nltk.corpus import wordnet as wn

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def lemmas_count(word,synset):
    lemmas = synset.lemmas()
    lemma_names = synset.lemma_names()
    count = 0
    for i in range(len(lemma_names)):
        if lemma_names[i].lower() == word:
            count += lemmas[i].count()
    return count

def most_freq_synset(word,synsets):
    freq_count = None
    mini = 0
    for synset in synsets:
        if mini < lemmas_count(word,synset):
            mini = lemmas_count(word,synset)
            freq_count = synset
    try:
        if freq_count == None:
            return word
    except:
        return str(freq_count).split("'")[1].split(".")[0]
    return word

def tokenization(sent):
    word_dict_sent = []
    try:
        token = stanford_tagger.tag(word_tokenizer.tokenize(sent.decode('utf-8')))
    except:
        token = stanford_tagger.tag(word_tokenizer.tokenize(sent))
    
    for word,tag in token:
        word = lemmatize(stemmer.stem(word.lower()))
        if tag in ["JJ","JJR","JJS"]:
            word = most_freq_synset(word,wn.synsets(word,pos=wn.ADJ))
            word_dict_sent.append(word)
        elif tag in ["RB","RBR","RBS"]:
            word =  most_freq_synset(word,wn.synsets(word,pos=wn.ADV))
            word_dict_sent.append(word)
        elif tag in ["NN","NNS","NNP","NNPS"]:
            word = most_freq_synset(word,wn.synsets(word,pos=wn.NOUN))
            word_dict_sent.append(word)
        elif tag in ["VB","VBD","VBG","VBN","VBP","VBZ"]:
            word = most_freq_synset(word,wn.synsets(word,pos=wn.VERB))
            word_dict_sent.append(word)
        elif tag in ["IN","CD","TO"]:
            word_dict_sent.append(word)
    return word_dict_sent


def get_corpus(doc):
    corpus = []
    for sent in doc:
        corpus.append(tokenization(sent))
    return corpus

In [ ]:
question_tfidf = [] 

for doc in dev:
    bm25Model = BM25(get_corpus(doc["sentences"]))
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    question_tfidf_doc = []
    for ques in doc["qa"]:
        scores = bm25Model.get_scores(tokenization(ques["question"]), average_idf)
        question_tfidf_doc.append(scores)
    question_tfidf.append(question_tfidf_doc)

In [ ]:
import operator
import copy
question_index_rank = copy.deepcopy(question_tfidf)

for i in range(len(question_index_rank)):
    for j in range(len(question_index_rank[i])):
        question = question_index_rank[i][j]
        rank = {}
        for index in range(len(question)):
            rank[index] = question[index]
        question = sorted(rank.items(), key=operator.itemgetter(1),reverse=True)
        question_index_rank[i][j] = []
        for index,value in question:
            question_index_rank[i][j].append(index)

In [ ]:
# def get_best_doc_num2(query):
#     query =  transformer.transform(vectorizer.transform(get_BOW(query)))
#     result={}
#     for x in range(term_matrix.shape[0]):
#          result[x]=cos_distance(query.toarray(),term_matrix[x].toarray())
            
#     minvalue=1
#     first=0
#     for item in result:
#         if minvalue > result[item]:
#             minvalue=result[item]
#             first=item     
#     del result[first]
    
#     minvalue=1
#     second=0
#     for item in result:
#         if minvalue > result[item]:
#             minvalue=result[item]
#             second=item     
#     return first,second

In [ ]:
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from scipy.spatial.distance import cosine as cos_distance

# vectorizer = DictVectorizer()
# transformer = TfidfTransformer(smooth_idf=False,norm=None)

# # store guessed question sentence no
# match_sent= [] #[[(best_match_sent_no,second_match_sent_no),...][second doc]]
# count = 0

# for dev_doc in tqdm(dev, desc='Extracting sentences from documents'):
#     count += 1
#     doc_match_sent = []
#     term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(dev_doc)))
#     for qa in dev_doc['qa']:
#         doc_match_sent.append(get_best_doc_num2(qa['question']))
#     match_sent.append(doc_match_sent)

In [ ]:
# cor_sent = []
# cor_num = 0
# total = 0
# for i in range(len(dev)):
#     cor_per_doc = []
#     for j in range(len(dev[i]["qa"])):
#         cor_per_doc.append(dev[i]["qa"][j]["answer_sentence"])
# #         total += 1
#         if dev[i]["qa"][j]["answer_sentence"] in question_index_rank[i][j][:1]:
#             cor_num += 1
#     cor_sent.append(cor_per_doc)
# print (cor_num+0.0)/total

In [ ]:
def first_n(question_index_rank,n):
    match_sent = []
    for i in range(len(question_index_rank)):
        match_sent.append([])
        for j in range(len(question_index_rank[i])):
            match_sent[i].append(question_index_rank[i][j][:n])
    return match_sent

match_sent = first_n(question_index_rank,4)

# Entity Extraction

In [ ]:
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('stanford-ner-2016-10-31/classifiers/english.muc.7class.distsim.crf.ser.gz',
               'stanford-ner-2016-10-31/stanford-ner.jar') 
# st = StanfordNERTagger('/Users/Luna/Downloads/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz',
#                '/Users/Luna/Downloads/stanford-ner/stanford-ner.jar') 

# tokenize sentence
ner_tag_1 = []
for i in range(len(dev)):
    doc_tag = []
    for j in range(len(dev[i]['sentences'])):
        sentence = dev[i]['sentences'][j]
        if '"' in sentence:
            sentence = sentence.replace('"',"")
        if sentence[:-1]==".":
            sentence = sentence[:-1]
        try:
            doc_tag.append(word_tokenizer.tokenize(sentence.decode("utf-8")))
        except:
            doc_tag.append(word_tokenizer.tokenize(sentence))
            
    doc_tag = st.tag_sents(doc_tag)
    ner_tag_1.append(doc_tag)

In [ ]:
symbol_list_discard = ["(",")","[","]"]
symbol_list = [",","-","/",".",u'\u2013',u'\u002C',u'\u002E',u'\u2215']

def hasSymbol(inputString):
    
    for char in symbol_list_discard:
        if char in inputString:
            return False
    for char in symbol_list:
        if char in inputString:
            return True
    return False

def hasNumbers(inputString):
    if any((char.isdigit() or ('$' in char)) for char in inputString):
        return True
    else:
        for word in word_tokenizer.tokenize(inputString):
            if word in ['one', 'two', 'three', 'four', 'five', 'six', 'seven',
                  'eight', 'nine', 'eleven', 'twelve', 'thirteen',
                  'fourteen', 'fifteen', 'sixteen',
                  'ten', 'twenty', 'thirty', 'forty', 'fifty', 'sixty',
                  'seventy', 'eighty', 'ninety', 'seventeen', 'eighteen',
                  'nineteen',
                  'nm','Hz','millions','million','hundred',u'\xb0C',u'\xb0F']:
                return True
    return False

In [ ]:
def combine_same_word(tag_list):
    for k in range(len(tag_list)):
        for i in range(len(tag_list[k])):
            j = 0
            while j < len(tag_list[k][i])-2:
                term,tag = tag_list[k][i][j]
                term_n,tag_n = tag_list[k][i][j+1]
                term_n2,tag_n2 = tag_list[k][i][j+2]
                tmp = term+term_n+term_n2
                
                if hasSymbol(tmp) and tmp in dev[k]["sentences"][i]:
                    if tag_n2 != 'O':
                        temp =  (tmp,tag_n2)
                    elif tag_n != 'O':
                        temp =  (tmp,tag_n)
                    elif tag != 'O':
                        temp = (tmp,tag)
                    else:
                        temp = (tmp,"OTHER")
                    tag_list[k][i][j] = temp
                    del tag_list[k][i][j+2]
                    del tag_list[k][i][j+1]
                    j -= 1
                j += 1

In [ ]:
def combine_entity(tag_list):
    for k in range(len(tag_list)):
        for i in range(len(tag_list[k])):
            j = 0
            while j < len(tag_list[k][i])-1:
                term,tag = tag_list[k][i][j]
                term_n,tag_n = tag_list[k][i][j+1]
                tmp = term+term_n
                
                if tag == tag_n and tag != "O" and term_n!="," and term_n !=";":
                    if tmp in dev[k]["sentences"][i]:
                        temp =  (tmp,tag)
                    else:
                        temp =  (term + " " + term_n,tag)
                    tag_list[k][i][j] = temp
                    del tag_list[k][i][j+1]
                    j -= 1
                j += 1

In [ ]:
# turn O and ORGANIZATION into OTHER; digit into NUMBER

def tune_other_and_number(tag_list):
    for i in range(len(tag_list)): # each document
        for j in range(len(tag_list[i])): # each sentence
            for k in range(len(tag_list[i][j])): # each question
                term,tag = tag_list[i][j][k] 
                if term!='' and len(term)>0 and (term,tag)!=tag_list[i][j][0] and tag == 'O' and term[0].isupper():
                    tag_list[i][j][k] = (term,"OTHER")
                if  hasNumbers(term) and (tag == "O" or tag =="OTHER"):
                    tag_list[i][j][k] = (term,"NUMBER")

In [ ]:
stopword = ["of"]
tags = ["LOCATION","ORGANIZATION","OTHER"]

def link_stopword(tag_list):
    for k in range(len(tag_list)):
        for i in range(len(tag_list[k])):
            j = 0
            while j < len(tag_list[k][i])-2:
                term,tag = tag_list[k][i][j]
                term_n,tag_n = tag_list[k][i][j+1]
                term_n2,tag_n2 = tag_list[k][i][j+2]
                tmp = term+" "+term_n+" "+term_n2
                
                if term_n in stopword and (tag in tags and tag_n2 in tags):
                    if tag_n2 != 'OTHER':
                        temp =  (tmp,tag_n2)
                    elif tag != 'OTHER':
                        temp = (tmp,tag)
                    else:
                        temp = (tmp,"OTHER")
                    tag_list[k][i][j] = temp
                    del tag_list[k][i][j+2]
                    del tag_list[k][i][j+1]
                    j -= 1
                j += 1

In [ ]:
import copy
ner_tag = []
ner_tag = copy.deepcopy(ner_tag_1)

combine_same_word(ner_tag)
combine_entity(ner_tag)
tune_other_and_number(ner_tag)
combine_entity(ner_tag)
link_stopword(ner_tag)

In [ ]:
tags = ["PERSON","LOCATION","NUMBER","OTHER","ORGANIZATION","PERCENT","DATE"]

def create_entity(first,second):
    sent_tag_dict = dict.fromkeys(tags,[])
    for k in [first]:
        for j in ner_tag[i][k]:
            term,tag = j
            if term =='' or len(term) == 1:
                continue
            if tag in tags:
                sent_tag_dict[tag] = sent_tag_dict[tag]+ [term]
            elif tag == "TIME":
                sent_tag_dict["NUMBER"] = sent_tag_dict["NUMBER"]+ [term]
    for tag in tags:
        sent_tag_dict[tag] = list(set(sent_tag_dict[tag]))
    return sent_tag_dict

def remove_tag(list_tag):
    list_tmp = []
    for tup in list_tag:
        term,tag = tup
        if term != '':
            list_tmp.append(term)
    return list_tmp
def remove_tag(list_tag):
    list_tmp = []
    for tup in list_tag:
        term,tag = tup
        if term != '':
            list_tmp.append(term)
    return list_tmp

In [ ]:
question_sent_list = []
entity_pool = []

for i in range(len(match_sent)):
    doc = dev[i]
    sent_pool = doc['sentences']
    test_list_doc = []
    entity_pool_doc = []
    
    for ques in match_sent[i]:
        test_list_sent = []
        entity_pool_sent = []
        for index in ques:
            test_list_sent.append(remove_tag(ner_tag[i][index]))
            entity_pool_sent.append(create_entity(index,i))
        test_list_doc.append(test_list_sent)
        entity_pool_doc.append(entity_pool_sent)
        
    question_sent_list.append(test_list_doc)
    entity_pool.append(entity_pool_doc)

# Answer Ranking

In [ ]:
import csv
import nltk
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
vectorizer = DictVectorizer()
svm = SVC(kernel='linear', C=1)

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

import ast

def train():
    f1 = open('output.csv')
    csv1 = csv.DictReader(f1)
    texts = []
    target = []

    for item in csv1:

        BOW = {}
        try:
            question = item["new"]
            questionx = ast.literal_eval(question)
            # print x[0]
            # print type(x)
            if questionx == [] or questionx == None:
                continue
            for word in questionx:
                BOW[word] = BOW.get(word, 0) + 1
            texts.append(BOW)
            target.append(item['Type'])
        except:
            continue

    brown_matrix = vectorizer.fit_transform(texts).toarray()
    svm.fit(brown_matrix, target)
train()
def predict(sentenct):
    BOW = {}
    for word in word_tokenizer.tokenize(sentenct):
        word=lemmatize(word)
        BOW[word] = BOW.get(word, 0) + 1
#     print vectorizer.transform([BOW])
    sentenctlist=vectorizer.transform([BOW]).toarray()
    return svm.predict(sentenctlist)

In [ ]:
import re
import nltk
import operator
from nltk import word_tokenize

def get_ranked_ans(entities_dic, question, sentence_token):
    # identify if the entity set is empty. If True, return nothing
    tmp_rank = {}
    is_empty = True
   
    for values in entities_dic.values():
        if len(values) != 0:
            is_empty = False
            
    q_type = predict(question)[0]
    if is_empty == False:
        resultTpye={"PERSON":{},"LOCATION":{},"NUMBER":{},"OTHER":{},"ORGANIZATION":{},"PERCENT":{},"DATE":{}}
        entitiesList=[]
        for ent_type,entities in entities_dic.items():
            for entity in entities:
                resultTpye[ent_type][entity]=0
                if (entity.lower() in question.lower()) or (entity.lower().replace('-', ' ') in question.lower()):
                    resultTpye[ent_type][entity] -= 999
                entitiesList.append(entity)
                
        preferred_entity = get_preferred_entity_list(entitiesList, sentence_token, question)
        if not preferred_entity==None and not preferred_entity== {}:
          
            maxvalue=max([i for i in preferred_entity.values()]) 
            if maxvalue!=0:
                for ent_type in resultTpye:
                    for word in resultTpye[ent_type]:
                        try:
                            resultTpye[ent_type][word] += round(preferred_entity[word]*1.00,2)/maxvalue
                        except:
                            continue

        if q_type =='PERSON':
            resultTpye=setWeight(resultTpye,[1,0.4,0.2,0.4,0.8,0.2,0.2])
                
        elif q_type =='ORGANIZATION':
            resultTpye=setWeight(resultTpye,[0.9,0.5,0.2,0.5,1,0.2,0.2])
        
        elif q_type =='LOCATION':
            resultTpye=setWeight(resultTpye,[0.8,1,0.2,0.5,0.7,0.2,0.2])
                
        elif q_type =='PERCENT':
            resultTpye=setWeight(resultTpye,[0.8,0.5,0.8,0.4,0.7,1,0.6])
                
        elif q_type =='DATE':
            
            resultTpye=setWeight(resultTpye,[0.8,0.5,0.5,0.5,0.7,0.7,1])
            for item in resultTpye['DATE']:
                if bool(re.match('(?<!\d)\d{4}[s]?(?!\d)', item)) and ('year' in question):
                    resultTpye['DATE'][item]+=1
                    
        elif q_type =='NUMBER':
            resultTpye=setWeight(resultTpye,[0.8,0.5,1,0.5,0.7,0.5,0.5])
            if ('percentage' in question) or ('percent' in question):
                 for item in resultTpye['PERCENT']:
                    resultTpye['PERCENT'][item]+=1
        
        # sort and choose the best answer
        sorted_ans = sorted(tmp_rank.items(), key=operator.itemgetter(1), reverse=True)
        
        # log for error analysis
#         output_file.write('Q_type: ' + '\t' + q_type + '\n')
#         output_file.write('Ranked Answers: ' + '\t' + str(resultTpye) + '\n\n')
      
        best=0
        best_ans=''
        for ent_type in resultTpye:
            for word in resultTpye[ent_type]:
                if resultTpye[ent_type][word] > best:
                    best_ans=word
                    best=resultTpye[ent_type][word]
        
        if q_type =='DATE' and ('year' in question):
            regex = re.compile(r'(?<!\d)\d{4}[s]?(?!\d)')
            year = regex.findall(best_ans)
            if year != []:
                best_ans = year[0]
                        
        return best_ans, best, entitiesList
    else:
        return '', 0, []

In [ ]:
def setWeight(resultTpye,weight):
            for item in resultTpye['PERSON']:
                resultTpye['PERSON'][item]+=weight[0]
            for item in resultTpye['LOCATION']:
                resultTpye['LOCATION'][item]+=weight[1]
            for item in resultTpye['NUMBER']:
                resultTpye['NUMBER'][item]+=weight[2]
            for item in resultTpye['OTHER']:
                resultTpye['OTHER'][item]+=weight[3]
            for item in resultTpye['ORGANIZATION']:
                resultTpye['ORGANIZATION'][item]+=weight[4]
            for item in resultTpye['PERCENT']:
                resultTpye['PERCENT'][item]+=weight[5]
            for item in resultTpye['DATE']:
                resultTpye['DATE'][item]+=weight[6]
            return resultTpye

In [ ]:
def get_question_type(question):
    # TODO: HAND-CODED, NEED TO BE REFINED!!
    # TODO: need to low-case to compare?

    result = predict(question)[0]
    q_type=''
    if result=='TIME':
        q_type=='NUMBER'
    else:
        q_type == result
#     print q_type
    return q_type

In [ ]:
def get_preferred_entity_list(entity_list, sentence_token, question):
    preferred_entity_list = []
    question_text = word_tokenize(question)
    sentence_tag = nltk.pos_tag(sentence_token)
    question_tag = nltk.pos_tag(question_text)
    
    # initialize a list for comparing, and set all elements as 0
    is_open_word = [0] * len(sentence_token)
    # find an open word in the question
    for word, tag in question_tag:
        if tag in ['JJ','JJR','JJS','FW','NN','NNS','NNPS','NNP','VBP','VB',
                   'VBG','VBN','VBZ','VBP','RB','RBR','RBS']:
            # if the open word appears in the sentence, then mark as 1
            for i in range(len(sentence_token)):
                if sentence_token[i] == word:
                    is_open_word[i] = 1
                    
# --------------------------------------------------------------------

    # find the entity which is closest to open-class words
    def get_distance(entity):
        # get the position of entity, and find the open class words
        covered_OCW = 0
        position = sentence_token.index(entity)
        
        # find number of covered open-class words in a given range
        #TODO: find the best window parameter
        length = len(sentence_token)/5
        window_min = position - length
        window_max = position + length 
        # when touch the start or the end of the sentence
        if window_min < 0:
            window_min = 0
            window_max += (0-window_min)
        if window_max > (len(sentence_token) - 1):
            window_min -= (window_max - len(sentence_token) + 1)
            window_max = (len(sentence_token) - 1)
        
        # get the total number of covered open-class words
        for i in range(window_min, window_max + 1):
            if is_open_word[i] == 1:            # find an open-class word
                covered_OCW += 1
        return covered_OCW

    # get distance for each entity and choose the best one
    all_distance = {}
    for entity in entity_list:
        try:
            all_distance[entity]=get_distance(entity)
        except:
            continue

    return all_distance

In [ ]:
# run on development data

with open("result.txt",'w') as output_file:
#     limit = 0
    all_count = 0
    correct_sum = 0
    corr_sen_retr_count = 0
    wrong_but_in_pool = 0
    partialy_correct = 0
#     no_entity_in_first_sent = 0

#     print '='*78
#     print 'Use top %d sentences: ' % (limit + 1)
    for i in range(len(match_sent)):
#     for i in tqdm(range(len(match_sent)), desc='Answering'):
        for j in range(len(match_sent[i])):
#             result = ''
#             backoff = 0
            all_count += 1
            cor_answer = dev[i]["qa"][j]['answer']
            Q = dev[i]["qa"][j]['question']
            A_sentence = dev[i]["sentences"][dev[i]["qa"][j]['answer_sentence']]

#             result = get_ranked_ans(entity_pool[i][j][0], Q, question_sent_list[i][j][0])
            guessed_sentence = dev[i]['sentences'][match_sent[i][j][0]]
            guessed_entities = entity_pool[i][j][0]

            
#             if result == '':
#                 no_entity_in_first_sent += 1
            
#             # If didn't got answer in current sentence, then backoff to next candidate sentence
#             # until getting the correct answer
            
#             while (result == '') and (backoff < limit):
#                 backoff += 1
#                 result = get_ranked_ans(entity_pool[i][j][backoff], Q, question_sent_list[i][j][backoff])
#                 guessed_sentence = dev[i]['sentences'][match_sent[i][j][backoff]]
#                 guessed_entities = entity_pool[i][j][backoff]

            result1, score1, pool1 = get_ranked_ans(entity_pool[i][j][0], Q, question_sent_list[i][j][0])
            result2, score2, pool2 = get_ranked_ans(entity_pool[i][j][1], Q, question_sent_list[i][j][1])
            
            if score2 * 0.2 > score1:
                result = result2
                if (result != cor_answer) and (result != '') and (result in pool2):
                    wrong_but_in_pool += 1
                if (result != cor_answer) and (result in cor_answer):
                    partialy_correct += 1
            else:
                result = result1
                if (result != cor_answer) and (result != '') and (result in pool1):
                    wrong_but_in_pool += 1
                if (result != cor_answer) and (result in cor_answer):
                    partialy_correct += 1
#             output_file.write('Retrieved Entities: ' + '\t' + str(guessed_entities) + '\n\n')

            if result == cor_answer:
                correct_sum += 1
                if A_sentence == guessed_sentence:
                    corr_sen_retr_count += 1
            else:
                guess_print = 'guessed sentence: ' + '\t' + guessed_sentence.encode('utf-8')+"\n\n"
                wrong_1 = '======== WRONG SENTENCES! ======== \n' + 'guessed sentence: ' + '\t' + guessed_sentence.encode('utf-8')+"\n\n"
                wrong_2 = 'CORRECT SENTENCE: ' + '\t' + A_sentence.encode('utf-8')+"\n\n"
                Q_print = 'Q: ' + '\t' + Q.encode('utf-8') + '\n\n'
                corr_ans_print = 'CORRECT ANSWER: ' + '\t' + cor_answer.encode('utf-8') + '\n'
                guessed_ans_print = 'guessed answer: ' + '\t' + result.encode('utf-8')+"\n"

                if A_sentence != guessed_sentence:
                    output_file.write(wrong_1)
                    output_file.write(wrong_2)
                else:
                    corr_sen_retr_count += 1
                    output_file.write(guess_print)
                output_file.write(Q_print)
                output_file.write('='*60 + '\n')
                output_file.write(corr_ans_print)
                output_file.write(guessed_ans_print)
                output_file.write('='*60 + '\n\n')

    print 'correct sum: ' + str(correct_sum)
    print 'Sentence Recall: ' + str((corr_sen_retr_count + 0.0) / all_count)
#     print "%d No.1 ranked sentences has no entity extracted at all." % no_entity_in_first_sent
#     print 'Wrong but in pool: ', wrong_but_in_pool / all_count
    print "Partialy correct: ", partialy_correct / all_count
    print "【 SCORE : " + str((correct_sum + 0.0) / all_count) + ' 】\n\n'

In [ ]:
# # run on test data

# with open("result_to_kaggle.txt",'w') as output_file:
# #     limit = 3
#     output_file.write('id,answer'+'\n')
#     for i in range(len(match_sent)):
#         for j in range(len(match_sent[i])):
#             result = ''
#             Q = dev[i]["qa"][j]['question']

#             result1,score1 = get_ranked_ans(entity_pool[i][j][0], Q, question_sent_list[i][j][0])
#             result2,score2 = get_ranked_ans(entity_pool[i][j][1], Q, question_sent_list[i][j][1])
            
#             if score2 * 0.2>score1:
#                 result=result2
#             else:
#                 result=result1
                
#             result = result.encode('utf-8')
#             reuslt = result.replace('" ','')
#             result = result.replace('"','')
#             result = result.replace(",","-COMMA-")
#             q_id = dev[i]["qa"][j]['id']
#             output_file.write(str(q_id) + ',' + str(result) + '\n')